In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing()
df = pd.DataFrame(california.data, columns=california.feature_names)
df['Price'] = california.target

for col in df.columns:
    df[col] = df[col].astype(str) 
    df[col] = df[col].str.replace(r"[\$,]", "", regex=True).str.strip()
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna()


In [2]:
X = df.drop('Price', axis=1)
y = df['Price']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

cat_model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    random_seed=42,
    verbose=0  
)
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)

print("\n📊 Cat Boost Performance:")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, y_pred_cat):.4f}")
print(f"R² Score: {r2_score(y_test, y_pred_cat):.4f}")

input_values = [8.3252, 41, 6.984, 1.023, 322, 2.5, 37.88, -122.23]
input_df = pd.DataFrame([input_values], columns=X.columns)

predicted_price = cat_model.predict(input_df)
print(f"\n💰 Predicted House Price (Cat Boosting): ${predicted_price[0]*100000:.2f}")






📊 Cat Boost Performance:
Mean Squared Error (MSE): 0.1922
R² Score: 0.8533

💰 Predicted House Price (Cat Boosting): $367240.66
